<a href="https://colab.research.google.com/github/rebeccahe1998/ScrapeingStuff/blob/main/Updated_of_Count_Call_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import random
import os
import re
import numpy as np
import pandas as pd
!pip install html2text
import html2text
!pip install yfinance
import yfinance as yf

1. Get ticker and its full name required for Motley Fool search url
2. Get all the available transcripts urls for the ticker
3. Count how many analysts of text after the key word by splitting keyword 'Call participants'
4. Generate a Excel file

In [2]:
def full_name(ticker):
    return yf.Ticker(ticker).info['longName'].split()[0].split(',')[0]
full_name('DAVA')

'Endava'

In [3]:

user_agent = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36",
                "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.10 (KHTML, like Gecko) Chrome/7.0.540.0 Safari/534.10",
                "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/534.4 (KHTML, like Gecko) Chrome/6.0.481.0 Safari/534.4",
                "Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-US) AppleWebKit/533.4 (KHTML, like Gecko) Chrome/5.0.375.86 Safari/533.4",
                "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/532.2 (KHTML, like Gecko) Chrome/4.0.223.3 Safari/532.2",
                "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.201.1 Safari/532.0",
                "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/3.0.195.27 Safari/532.0",
                "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/530.5 (KHTML, like Gecko) Chrome/2.0.173.1 Safari/530.5",
                "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/534.10 (KHTML, like Gecko) Chrome/8.0.558.0 Safari/534.10",
                "Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/540.0 (KHTML,like Gecko) Chrome/9.1.0.0 Safari/540.0"]

In [68]:
def headers():
    return {'User-Agent':user_agent[random.randint(0, len(user_agent) - 1)],}
def get_urls(full_name, ticker):
    url_ = []
    res = requests.get(f'https://www.fool.com/quote/nasdaq/{full_name}/{ticker}/',
                   headers=headers(),proxies={'http':'150.129.148.88'})
    # test which exchange(NYSE OR NASDAQ)
    if not res.status_code == 200:
        try:
            res = requests.get(f'https://www.fool.com/quote/nyse/{full_name}/{ticker}/',
                   headers=headers(),proxies={'http':'150.129.148.88'})
        except: print('URL might be wrong')
    soup = BeautifulSoup(res.content,'lxml')
    for i in soup.find_all('a', {'data-id' : 'article-list-hl'}, href=re.compile("transcripts")):
        url_.append('https://www.fool.com'+i.get('href'))
    # remove duplictes links 
    url_list = []
    url_date = []
    for url_i in url_:
        try:
            if re.split(r'(q\d+-\d+)',url_i)[1] not in url_date:
                url_date.append(re.split(r'(q\d+-\d+)',url_i)[1])
                url_list.append(url_i)
        
            else: pass
        except: pass
    return url_date,url_list # return 'q3-2020', and url
get_urls('WNS','WNS')

(['q2-2021', 'q1-2021', 'q4-2020', 'q3-2020', 'q2-2020', 'q1-2020', 'q4-2019'],
 ['https://www.fool.com/earnings/call-transcripts/2020/10/15/wns-holdings-limited-wns-q2-2021-earnings-call-tra/',
  'https://www.fool.com/earnings/call-transcripts/2020/07/16/wns-holdings-limited-wns-q1-2021-earnings-call-tra.aspx',
  'https://www.fool.com/earnings/call-transcripts/2020/04/23/wns-holdings-limited-wns-q4-2020-earnings-call-tra.aspx',
  'https://www.fool.com/earnings/call-transcripts/2020/01/16/wns-holdings-limited-wns-q3-2020-earnings-call-tra.aspx',
  'https://www.fool.com/earnings/call-transcripts/2019/10/17/wns-holdings-limited-wns-q2-2020-earnings-call-tra.aspx',
  'https://www.fool.com/earnings/call-transcripts/2019/07/18/wns-holdings-limited-wns-q1-2020-earnings-call-tra.aspx',
  'https://www.fool.com/earnings/call-transcripts/2019/04/25/wns-holdings-limited-wns-q4-2019-earnings-call-tra.aspx'])

In [56]:
def get_(full_name, ticker):
    url_ = []
    res = requests.get(f'https://www.fool.com/quote/nasdaq/{full_name}/{ticker}/',
                   headers=headers(),proxies={'http':'150.129.148.88'})
    # test which exchange(NYSE OR NASDAQ)
    if not res.status_code == 200:
        try:
            res = requests.get(f'https://www.fool.com/quote/nyse/{full_name}/{ticker}/',
                   headers=headers(),proxies={'http':'150.129.148.88'})
        except: print('URL might be wrong')
    soup = BeautifulSoup(res.content,'lxml')
    for i in soup.find_all('a', {'data-id' : 'article-list-hl'}, href=re.compile("transcripts")):
        url_.append('https://www.fool.com'+i.get('href'))
    return url_
get_('WNS','WNS')

['https://www.fool.com/earnings/call-transcripts/2020/10/15/wns-holdings-limited-wns-q2-2021-earnings-call-tra/',
 'https://www.fool.com/earnings/call-transcripts/2020/07/16/wns-holdings-limited-wns-q1-2021-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2020/04/23/wns-holdings-limited-wns-q4-2020-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2020/01/16/wns-holdings-limited-wns-q3-2020-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2019/10/17/wns-holdings-limited-wns-q2-2020-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2019/07/18/wns-holdings-limited-wns-q1-2020-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2019/04/25/wns-holdings-limited-wns-q4-2019-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2019/01/17/wns-holdings-shs-sponsored-american-deposit-share.aspx',
 'https://www.fool.com/earnings/call-transcripts/2020/10/15/w

In [57]:
kk= ['https://www.fool.com/earnings/call-transcripts/2020/10/15/wns-holdings-limited-wns-q2-2021-earnings-call-tra/',
 'https://www.fool.com/earnings/call-transcripts/2020/07/16/wns-holdings-limited-wns-q1-2021-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2020/04/23/wns-holdings-limited-wns-q4-2020-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2020/01/16/wns-holdings-limited-wns-q3-2020-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2019/10/17/wns-holdings-limited-wns-q2-2020-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2019/07/18/wns-holdings-limited-wns-q1-2020-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2019/04/25/wns-holdings-limited-wns-q4-2019-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2019/01/17/wns-holdings-shs-sponsored-american-deposit-share.aspx',
 'https://www.fool.com/earnings/call-transcripts/2020/10/15/wns-holdings-limited-wns-q2-2021-earnings-call-tra/',
 'https://www.fool.com/earnings/call-transcripts/2020/07/16/wns-holdings-limited-wns-q1-2021-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2020/04/23/wns-holdings-limited-wns-q4-2020-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2020/01/16/wns-holdings-limited-wns-q3-2020-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2019/10/17/wns-holdings-limited-wns-q2-2020-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2019/07/18/wns-holdings-limited-wns-q1-2020-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2019/04/25/wns-holdings-limited-wns-q4-2019-earnings-call-tra.aspx',
 'https://www.fool.com/earnings/call-transcripts/2019/01/17/wns-holdings-shs-sponsored-american-deposit-share.aspx']

In [67]:
url_date=[]
url_list=[]
try:
    if re.split(r'(q\d+-\d+)',kk[7])[1] not in url_date:
        url_date.append(re.split(r'(q\d+-\d+)',url_i)[1])
        url_list.append(kk[7])

    else: pass
except: 
    pass
url_list

[]

In [5]:
def get_call_participated_analyst(ticker):
    full = full_name(ticker) 
    url_date, url_list = get_urls(full,ticker)
    
    inner_url_analyst_count = {'Date':[], ticker: []}

    for i,(inner_url, inner_date) in enumerate(zip(url_list,url_date)):
        time.sleep(np.random.randint(10))
        try: 
            res = requests.get(inner_url,headers=headers(),proxies={'http':'150.129.148.88'})

            inner_url_analyst_count['Date'].append(inner_date) # append date
        except: print('Requests went wrong')

        soup = BeautifulSoup(res.content,'lxml') 
        text = str(res.text)
        text = html2text.html2text(text)
        for i in ['Call Participants','Call participant']:
            text = text.replace(i, 'Call participants')
        participants = text.split('Call participants')[-1] #split by keyword to get participants
        inner_url_analyst_count[ticker].append(len(re.findall(r'Analyst|Unidentified',participants)))
        
    return inner_url_analyst_count

In [6]:
get_call_participated_analyst('DAVA')

{'DAVA': [9, 6, 8, 4, 5, 6, 5],
 'Date': ['q3-2020',
  'q2-2020',
  'q1-2020',
  'q4-2019',
  'q3-2019',
  'q1-2019',
  'q4-2018']}

In [8]:

def get_df(ticker):
    df = pd.DataFrame.from_dict(get_call_participated_analyst(ticker), orient='index').reindex(['Date',ticker])
    df.columns = df.iloc[0]
    df = df[1:]
    return df

def merge_df(ticker_list):
    df = get_df(ticker_list[0])
    try:
        for ticc in ticker_list[1:]:
            to_join = get_df(ticc)
            df = pd.concat([df,to_join])
    except: pass
    return df



In [9]:
# trying the code above
ticker_list = ['VNET', 'DAVA']#, 'FROG', 'LOPE', 'CCMP', 'MSGS', 'SONO', 'DOYU', 'WEN']
joined = merge_df(ticker_list)

In [10]:
joined # looks good

,q3-2020,q2-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018,q2-2018
VNET,9,9,5,6,4,3,2,5,7
DAVA,9,6,8,4,5,NaN,6,5,NaN


In [18]:
df = pd.read_excel('scrape_list.xlsx') # read in the ticker excel file
mylist = df['Ticker'].tolist()
ticker_list = mylist[0:10]
ticker_list


['TWOU',
 'DRVN',
 'VNET',
 'DAVA',
 'FROG',
 'LOPE',
 'CCMP',
 'MSGS',
 'SONO',
 'DOYU']

In [19]:
joined = merge_df(ticker_list)

In [20]:
joined

,q4-2020,q3-2020,q2-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018,q3-2018,q2-2018,q1-2021
TWOU,8,7,6,6,6,7,11,10,12,15,NaN,NaN
DRVN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VNET,NaN,9,9,5,6,4,3,2,5,NaN,7,NaN
DAVA,NaN,9,6,8,4,5,NaN,6,5,NaN,NaN,NaN
FROG,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOPE,4,4,3,3,3,1,2,3,3,NaN,NaN,NaN
CCMP,NaN,NaN,7,6,6,5,3,4,4,NaN,NaN,6
MSGS,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SONO,7,6,7,7,8,5,6,5,NaN,4,NaN,6
DOYU,NaN,NaN,NaN,6,5,6,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
writer = pd.ExcelWriter('Earnings_Call_Analyst_Count.xlsx')
joined.to_excel(writer,header = True, index=True)
writer.save()

In [22]:
ticker_list = mylist[10:20]
joined = merge_df(ticker_list)
joined

,q4-2020,q3-2020,q2-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018,q3-2018,q2-2018,q2-2021,q4-2017,q3-2021,q1-2021,q4-2021
WEN,20,17,15,18,18,13,17,13,20,13,NaN,NaN,NaN,NaN,NaN,NaN
SGMS,6,NaN,NaN,5,6,5,6,6,7,NaN,4,NaN,NaN,NaN,NaN,NaN
SYNA,NaN,2,5,6,3,5,5,6,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN
SNX,7,7,7,7,6,7,6,NaN,5,6,7,NaN,NaN,NaN,NaN,NaN
ALTR,8,7,6,6,6,NaN,6,7,4,NaN,NaN,NaN,2,NaN,NaN,NaN
M,11,13,13,9,16,15,13,13,12,13,NaN,NaN,NaN,NaN,NaN,NaN
FEYE,11,9,11,10,10,10,10,12,10,12,NaN,NaN,NaN,NaN,NaN,NaN
CRUS,8,7,10,8,7,NaN,7,NaN,NaN,8,NaN,6,NaN,9,10,NaN
NCR,8,9,7,10,9,8,7,5,8,7,NaN,NaN,NaN,NaN,NaN,NaN
SMTC,3,10,12,12,5,4,NaN,NaN,NaN,NaN,NaN,12,NaN,10,12,10


In [24]:
ticker_list = mylist[20:50]
joined = merge_df(ticker_list)
joined

,q2-2021,q1-2021,q4-2020,q3-2020,q2-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018,q3-2018,q4-20,q3-2021,q2-2018,q4-2017,q4-2021
PRSP,4,3,5,3,5,3,6,5,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SABR,NaN,NaN,5,5,5,4,NaN,NaN,7,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
APPF,NaN,NaN,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
VERX,NaN,NaN,9,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LESL,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BIGC,NaN,NaN,13,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MPLN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JAMF,NaN,NaN,8,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JOBS,NaN,NaN,NaN,NaN,3,4,4,3,2,NaN,2,3,NaN,NaN,NaN,NaN,NaN


In [26]:
joined.to_excel(writer,header = True, index=True)
writer.save()

In [29]:
ticker_list = mylist[50:60]
joined = merge_df(ticker_list)
joined

,q4-2020,q3-2020,q2-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018,q3-2021,q2-2021,q1-2021,q3-2018
JMIA,2,7,3,3,5,5,7,NaN,NaN,NaN,NaN,NaN,NaN
AMC,6,7,5,6,4,6,3,3,4,NaN,NaN,NaN,NaN
MSP,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TIGO,7,7,6,8,8,6,6,NaN,NaN,NaN,NaN,NaN,NaN
NEWR,11,14,15,13,12,13,NaN,NaN,NaN,10,8,10,NaN
INOV,NaN,NaN,NaN,6,5,5,1,6,6,NaN,NaN,NaN,NaN
BZUN,8,4,8,10,9,8,9,8,4,NaN,NaN,NaN,8
TGNA,8,NaN,NaN,7,7,10,9,5,10,NaN,NaN,NaN,8
SIX,12,9,10,10,9,8,11,9,9,NaN,NaN,NaN,9
EYE,7,11,8,9,9,10,10,8,8,NaN,NaN,NaN,7


In [32]:
ticker_list = mylist[60:65]
joined = merge_df(ticker_list)
joined


,q4-2020,q3-2020,q2-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018,q3-2018,q4-2021,q3-2021,q2-2021,q1-2021
DDD,8,8,4,7,6,6,7,8,4,6,NaN,NaN,NaN,NaN
AVT,NaN,9,9,NaN,4,8,9,6,NaN,NaN,NaN,NaN,NaN,NaN
QLYS,9,NaN,9,4,8,11,12,11,11,NaN,NaN,NaN,NaN,NaN
PD,7,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,8,5,6
SWCH,13,8,6,8,6,7,8,8,8,7,NaN,NaN,NaN,NaN


In [39]:
ticker_list = mylist[65:67]
joined = merge_df(ticker_list)
joined

,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018,q4-2020,q3-2018,q2-2018,q1-2018,q4-2017
SEAS,9,6,8,9,7,6,NaN,NaN,NaN,NaN,NaN
KBH,8,8,11,9,NaN,NaN,7,9,10,4,11


In [41]:
ticker_list = mylist[67:69]
joined = merge_df(ticker_list)
joined

Date,q4-2020,q3-2020,q2-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018
MDC,6,8,9,8,10,8,7,9,10
IGT,5,6,6,7,7,6,8,6,7


In [70]:
ticker_list = mylist[69:80]
joined = merge_df(ticker_list)
joined

,q2-2021,q1-2021,q4-2020,q3-2020,q2-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018,q2-2018,q3-2021,q3-2018
WNS,9,10,10,12,11,10,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VIAV,8,NaN,NaN,6,7,7,9,6,7,10,10,NaN,NaN,NaN
CDLX,NaN,NaN,5,7,7,8,6,6,3,5,6,4,NaN,NaN
CBRL,8,7,8,7,7,8,6,9,6,7,NaN,NaN,NaN,NaN
BAND,NaN,NaN,NaN,NaN,NaN,7,7,5,6,4,6,NaN,NaN,NaN
EGHT,14,11,8,10,9,12,11,12,11,NaN,NaN,NaN,NaN,NaN
MMYT,1,0,2,1,3,NaN,3,3,NaN,2,NaN,NaN,1,NaN
CRI,NaN,NaN,4,8,9,5,5,9,11,8,6,NaN,NaN,7
ENV,NaN,NaN,7,7,5,7,6,8,0,9,8,NaN,NaN,NaN
RRR,NaN,NaN,NaN,NaN,NaN,9,9,7,7,8,NaN,NaN,NaN,NaN


In [75]:
ticker_list = mylist[80:81]
joined = merge_df(ticker_list)
joined

Date,q1-2020,q4-2019,q3-2019,q2-2019
PSN,9,7,7,6


In [77]:
ticker_list = mylist[81:83]
joined = merge_df(ticker_list)
joined

,q4-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018,q3-2021,q2-2021,q1-2021,q3-2020,q2-2020
TMHC,8,8,9,5,11,10,10,NaN,NaN,NaN,NaN,NaN
VSAT,7,4,5,6,3,NaN,NaN,8,3,7,4,5


In [80]:
ticker_list = mylist[83:84]
joined = merge_df(ticker_list)
joined

Date,q4-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018
SPSC,9,8,8,10,7,8,10


In [82]:
ticker_list = mylist[85:86]
joined = merge_df(ticker_list)
joined

Date,q4-2020,q3-2020,q2-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018,q3-2018
STMP,3,2,3,4,3,4,5,5,5,5


In [84]:
ticker_list = mylist[86:100]
try:
    joined = merge_df(ticker_list)
except: pass
joined

,q4-2020,q3-2020,q2-2020,q1-2020,q4-2019,q3-2019,q2-2019,q1-2019,q4-2018,q3-2018,q1-2021,q2-2018
SPT,11,10,8,6,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FORM,6,6,7,NaN,7,6,5,5,4,5,NaN,NaN
DAN,7,9,9,10,10,9,10,4,8,6,NaN,NaN
ADNT,4,5,6,4,4,4,4,6,0,NaN,4,NaN
MOMO,NaN,2,3,3,2,4,4,3,4,4,NaN,3
LCII,8,9,7,8,7,7,5,10,6,NaN,NaN,NaN
DIOD,5,4,4,4,3,2,3,2,4,NaN,NaN,NaN
HGV,5,5,4,5,4,5,5,6,6,NaN,NaN,NaN
BECN,NaN,NaN,11,NaN,9,12,11,10,8,NaN,NaN,NaN
ICTPU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
writer = pd.ExcelWriter('_Count.xlsx')
joined.to_excel(writer,header = True, index=True)
writer.save()